# Providing access to Google drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Installing dependencies

In [ ]:
!pip install langchain langchain_core langchain-groq langchain-community pypdf langchain-text-splitters langchain-chroma langchain-huggingface sentence-transformers langsmith streamlit

# Document Ingestion

Load PDF content as Markdown

In [5]:
from langchain_community.document_loaders import PyPDFLoader

loader=PyPDFLoader("/content/drive/MyDrive/content/company_policy.pdf")
pages = loader.load()
text_content = "\n".join([page.page_content for page in pages])

Define the Text Splitter

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,          # Max size of each chunk (in characters)
    chunk_overlap=200,        # Number of characters to overlap between chunks
    length_function=len,      # Use Python's len() to measure length
)

chunks = text_splitter.split_documents(pages)

## Embedding and Storing Chunks

In [9]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma


# Initialize the Embedding Model
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Create the Vector Database
vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory="./architecture_db"
)

print("Vector database created and saved successfully!")

Vector database created and saved successfully!


## LCEL Retrieval Chain

Creating a prompt template

In [11]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Create the Prompt Template
template = """
You are a helpful HR assistant who knows all internal company policies.
Refer to each section of the document to answer the user's question.

If you don't know the answer based on the context, just say you don't know.
Keep the answer concise and professional.

Context:
{context}

Question:
{question}

Answer:
"""
prompt = ChatPromptTemplate.from_template(template)

Defining a Retriever

In [12]:
retriever = vector_db.as_retriever(search_kwargs={"k": 3})

# Observability with LangSmith tool

In [13]:
import os
from google.colab import userdata

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_API_KEY"] = userdata.get('LANGSMITH_API_KEY')
os.environ["LANGSMITH_PROJECT"] = "RAG_pipeline"
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

In [ ]:
import os
from google.colab import userdata
from langsmith import traceable
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, SystemMessage

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
    api_key=userdata.get("GROQ_API_KEY"),
    streaming=True
)

@traceable
def format_prompt(subject):
    """Converts a raw subject into a structured list of messages."""
    system_msg = SystemMessage(content="You are a helpful HR assistant who knows all internal company policies. Provide concise answers.")
    human_msg = HumanMessage(content=f"Extract the necessary information from a particular section: {subject}")
    return [system_msg, human_msg]

@traceable(run_type="llm")
def invoke_llm(messages):
    """Sends messages to LLM and returns the full response object."""
    response = llm.invoke(messages)
    return response

@traceable
def parse_output(response):
    """Extracts the text content from the LLM's complex response object."""
    return response.content.strip()

@traceable
def run_pipeline(subject):
    """Orchestrates the flow from input to final answer."""
    messages = format_prompt(subject)
    response = invoke_llm(messages)
    return parse_output(response)

Recording response in the notebook

In [28]:
result=run_pipeline("With reference to the Conditions of Employment, under what circumstances will the overtime of an employee be considered valid?")
print(f"Answer: {result}")

Answer: According to our company's Conditions of Employment (Section 5.3), overtime will be considered valid under the following circumstances:

- The overtime is approved in advance by the employee's supervisor or manager.
- The overtime is necessary to meet business needs or to complete a project deadline.
- The employee has been given adequate notice of the overtime requirement.
- The overtime does not exceed 10 hours per week, unless otherwise approved by HR.
- The employee is paid at a rate of 1.5 times their regular hourly rate for all overtime worked.

Please note that these conditions may be subject to change, and employees should refer to the most up-to-date version of the Conditions of Employment for the latest information.
